In [83]:
%matplotlib inline

import os,sys

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
from torch.autograd import Variable

import timeit

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def apply_mirror_boundary_conditions(coord, dim):
    """
    Return the correct coordinate according to mirror boundary conditions
        coord: a coordinate (x or y) in the image
        dim: the length of the axis of said coordinate
    """
    # If the coordinate is outside of the bounds of the axis, take its reflection inside the image
    if coord < 0:
        coord = -coord
    elif coord >= dim:
        coord =  2*(dim-1) - coord % (2*(dim-1))
    # Else, do nothing
    return int(coord)

def get_window(image, window_size, centre_coordinates):
    """
    Get a window in image taking into account boundary conditions
        image: a numpy array representing our image
        window_size: an odd number specifying the size of the window
        centre_coordinates: a list containing the x-y coordinates of the window's central pixel
    """
    # Get convenient variables
    window_radius = (window_size - 1)/2
    i_centre, j_centre = (centre_coordinates[0], centre_coordinates[1])
    nrows, ncols = image.shape
    window = np.zeros((window_size, window_size))
    # Fill in the window array with pixels of the image
    for i in range(window_size):
        # Apply mirror boundary conditions on the x-coordinate
        i_mirrored = apply_mirror_boundary_conditions(i_centre + i - window_radius, nrows)
        for j in range(window_size):
            # Same for the y-coordinate
            j_mirrored = apply_mirror_boundary_conditions(j_centre + j - window_radius, ncols)
            # Fill in the window with the corresponding pixel
            window[i, j] = image[i_mirrored, j_mirrored]
    return window

In [92]:
def shift_to_the_right(image, window, centre_coordinates):
    nrows, ncols = image.shape
    window_size = len(window)
    window_radius = (window_size - 1)/2
    j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + 1 + window_radius, ncols)
    shifted = np.roll(window, -1, axis=1)
    for i in range(window_size):
        i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + i - window_radius, nrows)
        shifted[i, -1] = image[i_mirrored, j_mirrored]
    return shifted

def shift_to_the_bottom(image, window, centre_coordinates):
    nrows, ncols = image.shape
    window_size = len(window)
    window_radius = (window_size - 1)/2
    i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + 1 + window_radius, nrows)
    shifted = np.roll(window, -1, axis=0)
    for j in range(window_size):
        j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + j - window_radius, ncols)
        shifted[-1, j] = image[i_mirrored, j_mirrored]
    return shifted

def sliding_window(image, window_size):
    """
    Construct a list of sliding windows of given size on an image.
    The windows will slide from left to right and from up to down.
        image: a numpy array representing our image
        window_size: an odd number specifying the size of the window
    """
    nrows, ncols = image.shape
    windows = []
    i = 0
    row_windows = [get_window(image, window_size, [i, 0])]
    for j in range(ncols-1):
        row_windows += [shift_to_the_right(image, row_windows[-1], [i, j])]
    windows += row_windows
    for i in range(nrows-1):
        row_windows = [shift_to_the_bottom(image, row_windows[j], [i, j]) for j in range(ncols)]
        windows += row_windows
    return windows

In [63]:
## Tests

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(a)
w = get_window(a, 3, [0,1])
print(w)
w_right = shift_to_the_right(a, w, [0,1])
print(w_right)
w_bottom = shift_to_the_bottom(a, w, [0,1])
print(w_bottom)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[ 4.  5.  6.]
 [ 1.  2.  3.]
 [ 4.  5.  6.]]
[[ 5.  6.  5.]
 [ 2.  3.  2.]
 [ 5.  6.  5.]]
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]]


In [73]:
ww = sliding_window(a, 3)
for w in ww:
    print(w)
    print('\n')

[[ 5.  4.  5.]
 [ 2.  1.  2.]
 [ 5.  4.  5.]]


[[ 4.  5.  6.]
 [ 1.  2.  3.]
 [ 4.  5.  6.]]


[[ 5.  6.  5.]
 [ 2.  3.  2.]
 [ 5.  6.  5.]]


[[ 2.  1.  2.]
 [ 5.  4.  5.]
 [ 8.  7.  8.]]


[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]]


[[ 2.  3.  2.]
 [ 5.  6.  5.]
 [ 8.  9.  8.]]


[[ 5.  4.  5.]
 [ 8.  7.  8.]
 [ 5.  4.  5.]]


[[ 4.  5.  6.]
 [ 7.  8.  9.]
 [ 4.  5.  6.]]


[[ 5.  6.  5.]
 [ 8.  9.  8.]
 [ 5.  6.  5.]]




In [76]:
# Directory and files name
train_dir = "training/"
tr_image_dir = train_dir + "images/"
tr_label_dir = train_dir + "groundtruth/"

tr_image_files = os.listdir(tr_image_dir)
tr_label_files = os.listdir(tr_label_dir)

# Number of training samples
N = len(tr_image_files)

# Load the images and ground truth
img_train = []
label_train = []
for i in range(N):
    img = mpimg.imread(tr_image_dir + tr_image_files[i])
    label = mpimg.imread(tr_label_dir + tr_label_files[i])
    
    img_train.append(img)
    label_train.append(label)

# Keep only sub-set of images
NUM_IMAGES = 5

img_train = np.asarray(img_train[:NUM_IMAGES])
label_train = np.asarray(label_train[:NUM_IMAGES])

print(img_train.shape, label_train.shape)

(5, 400, 400, 3) (5, 400, 400)


In [89]:
im = label_train[0]
for window_size in [3, 5, 7, 15, 35, 51]:
    %timeit sliding_window(im, window_size)

15.6 s ± 43.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
17.2 s ± 58.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
18.8 s ± 73.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
25.1 s ± 55.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
43.1 s ± 64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4min ± 1.46 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
def aa(image, window_size):
    """
    Construct a list of sliding windows of given size on an image.
    The windows will slide from left to right and from up to down.
        image: a numpy array representing our image
        window_size: an odd number specifying the size of the window
    """
    nrows, ncols = image.shape
    windows = []
    for i in range(nrows):
        for j in range(ncols):
            windows += [get_window(image, window_size, [i, j])]
    return windows

In [93]:
for window_size in [3, 5, 7, 15, 35, 51]:
    %timeit aa(im, window_size)

10.8 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
23.2 s ± 77.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
42.7 s ± 987 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2min 58s ± 11.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
14min 26s ± 2.84 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


KeyboardInterrupt: 